<a target="_blank" href="https://colab.research.google.com/github/robgen/HEDSpython/blob/main/Tutorial_6.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

#**Unsupervised Learning: PCA and k-means clustering**

In this tutorial we will go over two applications of unsupervised learning Principal Component Analysis and K-Means Clustering.

Unsupervised learning is a type of machine learning that learns from data without human supervision. Results from unsupervised learning usually require interpretation ex-post.

Let's import our usual libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

And load our csv

In [ ]:
csvFilePath = 'https://raw.githubusercontent.com/robgen/HEDSpython/refs/heads/main/files/WVS_Cross-National_Wave_7_csv_v4_0.csv'
rawData = pd.read_csv(csvFilePath)

In [ ]:
 rawData

Let's check what the initial shape of our dataframe is.

In [ ]:
print('the initial shape of the dataset is:', rawData.shape)

 Now let's take a look at the column names.

In [ ]:
print(rawData.columns.tolist())

To select the features which we need in our analysis we take a look at the *WVS questionnaire.pdf* file in th GitHub repository *files* folder to identify the relevant questions.

In this case, we are interested in the 290 questions of the main survey. We know these are registered as *Q* + *question number* so we proceed to subset the dataframe to contain only the questions.

In [ ]:
Nquestions = 290
featuresToKeep = []
for q in range(1,Nquestions+1):
    featuresToKeep.append('Q'+str(q))

WVS = rawData.loc[:, featuresToKeep]

print('the final shape of the dataset is:', WVS.shape)

In [ ]:
WVS

We also want to retain some general features for later.

In [ ]:
generalFeatures = ['B_COUNTRY_ALPHA', 'O1_LONGITUDE', 'O2_LATITUDE']
WVSgeneral = rawData.loc[:, generalFeatures]

We can now delete the del `rawData` from the local memory (as we won't need it anymore and it takes up quite a lot of memory!).

In [ ]:
del rawData

To get familiar with the data, let's plot a histogram of the observations by country.

In [ ]:
countries, numSurveysInCountry = np.unique(WVSgeneral.B_COUNTRY_ALPHA,return_counts=True)
dummy = range(len(countries))

plt.figure(dpi=200)
plt.rcParams.update({'font.size': 6})
plt.bar(dummy,numSurveysInCountry, align='center')
plt.xticks(dummy, countries)
plt.xticks(rotation = 90)
plt.show()

Now let's clean the dataset. For each question we want to check which is the most common value and how many null values there are.

In [ ]:
nanQuestions = []
weirdQuestions =[]
for q in range(1,Nquestions+1):
    labels, counts = np.unique(WVS['Q'+str(q)],return_counts=True)
    print('Q'+str(q)+' - ', 'Most common: ' , labels[counts.argmax()], ',',
            '#Empty: ', WVS['Q'+str(q)].isna().sum())
    if np.isnan(labels[counts.argmax()]):
        nanQuestions.append('Q' + str(q)) # append questions for which the most common value is nan
    if labels[counts.argmax()] > 10 or labels[counts.argmax()] < 0:
        weirdQuestions.append('Q'+str(q)) # append questions for which the most common value is above 10 or below 0

In [ ]:
nanQuestions

In [ ]:
weirdQuestions

We can use the questionnaire pdf in the files folder to check weird values. Q261 is the year of birth, Q262 is age so it is likely these values are above 10 (i.e., they do not look weird), while Q266 and Q272 are respecively out of 2 and 5 in the pdf, so they should not take values above 10 (i.e., they look weird). We re-define weird values accordingly.

In [ ]:
weirdQuestions = ['Q266', 'Q272']

We can then delete questions for which most common values are null or weird.

In [ ]:
questionsToRemove = nanQuestions + weirdQuestions
WVS = WVS.drop(questionsToRemove, axis=1)

And then substitute `NaN` values with the most frequent answer for each question (the mode). PCA cannot be applied to a dataset with null values.

In [ ]:
for name, values in WVS.items():
    labels, counts = np.unique(values,return_counts=True)
    values[values.isna()] = labels[counts.argmax()]

##**Principal Component Analysis**

We use the `sklearn` library to run our analysis. `sklearn` (or scikit-learn) is a powerful library for machine learning.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

Principal component analysis (PCA) is a dimensionality reduction technique that allows us to reduce a large set fo variables into a smaller one (called principal components) that still contains most of the information of the large dataset.

As a first step we need to standardize all variables in the dataset to make sure that each of the variables contribute equally in the analysis.
Standardizing a variable means to reduce it to a unit scale (that is to mean = 0 and variance = 1).

In [ ]:
standWVS = StandardScaler().fit_transform(WVS)

We use the `PCA()` and `.fit_transform()` functions to preform the reduction into a lower dimensional space, the argument `n_components` indicates how many components the variables should be reduced to.

To start with, let's reduce the variables to two principal components.

In [ ]:
pcaObj = PCA(n_components=2)
prComp = pcaObj.fit_transform(standWVS)

Let's take a look at the two components.

In [ ]:
plt.figure(dpi=100)
plt.scatter(prComp[:,0], prComp[:,1])
plt.xlabel('PC1 [-] VarExp=%1.2f' %pcaObj.explained_variance_ratio_[0])
plt.ylabel('PC2 [-] VarExp=%1.2f' %pcaObj.explained_variance_ratio_[1])
plt.show()

PC1 explains the variation on the x-axis while PC2 explains the variation on the y-axis. From this plot we can see already that PC1 explains more variance in the original dataset than PC2 as the scatter is more spread along the x-axis than the y-axis.

Let's now take a look at the loadings for each component.

In [ ]:
df_pca = pd.DataFrame(pcaObj.components_.T, columns=['PC1', 'PC2'])
print(df_pca)

We can use the largest and smallest loadings to interpret our components.

In [ ]:
print(df_pca['PC1'].nlargest(2))
print(df_pca['PC1'].nsmallest(2))

**PC1 interpretation**

At the *higher end* of the spectrum sit respondents that declared that 'Stealing property' (Q179) and 'Prostitution' (Q183) can be morally justified, that completely disagree with the statement 'One of the bad effects of science is that it breaks down people’s ideas of right and wrong' (Q161) and that have done the action 'Encouraging others to take action about political issues' (Q215).

While at *lower end* of the spectrum stand people who do not morally justify 'Stealing property' (Q179) and 'Prostitution' (Q183), that agree that 'One of the bad effects of science is that it breaks down people’s ideas of right and wrong' (Q161) and that would never 'encourage others to take action about political issues' (Q215).

What would you say is this PC measuring? Take a guess, there is no right or wrong answer.


In [ ]:
print(df_pca['PC2'].nlargest(2))
print(df_pca['PC2'].nsmallest(2))

**PC2 interpretation**

At the *higher end* of the spectrum are people who do not have a lot of confidence in 'the court' (Q70) and in 'political parties' (Q72) and that think that 'The state makes people’s incomes equal' (Q247) and 'Civil rights protect people from state oppression' (Q246) are not essential characteristics of democracy.

To the contrary, people with *lower scores* of PC2 will have a lot of confidence in 'the court' (Q70) and in 'political parties' (Q72) and will think that 'The state makes people’s incomes equal' (Q247) and 'Civil rights protect people from state oppression' (Q246) are more essential characteristics of democracy.

How do you interpret this PC?

The rule of thumb for PCA is that to have a good summary of the data the cumulative explained variance needs to exceed 70-80% of the variance.

Let's check how much variance is explained by PC1 and PC2.

In [ ]:
pcaObj.explained_variance_ratio_.cumsum()

Only 12%! Not a lot. Let's try with three components then.

In [ ]:
pcaObj = PCA(n_components=3)
prComp = pcaObj.fit_transform(standWVS)

In [ ]:
pcaObj.explained_variance_ratio_.cumsum()

Three PCs explain only 16% of the variance, that's not a lot either. We can still plot this to see what adding a third dimension looks like.

In [ ]:
fig = plt.figure(1, figsize=(4, 3), dpi=200)
plt.clf()

ax = fig.add_subplot(111, projection="3d", elev=48, azim=134)
ax.set_position([0, 0, 0.95, 1])
plt.cla()
ax.scatter(prComp[:,0], prComp[:,1], prComp[:,2], edgecolor="k")
ax.set_xlabel('PC1 [-] VarExp=%1.2f' %pcaObj.explained_variance_ratio_[0])
ax.set_ylabel('PC2 [-] VarExp=%1.2f' %pcaObj.explained_variance_ratio_[1])
ax.set_zlabel('PC3 [-] VarExp=%1.2f' %pcaObj.explained_variance_ratio_[2])
plt.show()

To see how many PCs we should reduce our sample to, let's calculate the percentage of variance explained by PCs from 2 up to 139 (half of our sample) - this is called sensitivity analysis.

In [ ]:
pcaObjDummy = PCA(n_components=139)
pcDummy = pcaObjDummy.fit_transform(standWVS)
VarianceExplained = pcaObjDummy.explained_variance_
totVarianceExplained = pcaObjDummy.explained_variance_ratio_.cumsum()

And now let's check how 139 PCs do.

In [ ]:
totVarianceExplained

Another way to pick the number of components is by using the *elbow method*.
That is, we plot the number of PCs over the total variance and look for an *elbow* (the place where the explained variation begins to slow) in the plot to indicate how many PCs is appropriate to use.

In [ ]:
plt.figure(dpi=200)
plt.plot(range(1, 140), VarianceExplained, marker='o')
plt.title('Sensitivity Analysis for PCA')
plt.xlabel('Number of PCs')
plt.ylabel('Variance [-]')
plt.show()

We can see a *elbow* around the 5th component indicating a decrease in the explained variation. Let's take a look more closely.

In [ ]:
plt.figure(dpi=200)
plt.plot(range(1, 11), VarianceExplained[:10], marker='o')
plt.title('Sensitivity Analysis for PCA')
plt.xlabel('Number of PCs')
plt.ylabel('Eigenvalue (variance) [-]')
plt.show()

In [ ]:
totVarianceExplained[5]

There is clearly a elbow in our plot. Yet, total variance explained by PC5 is only around 20%. Let's see how many components we may need to hit the 70-80% mark.

In [ ]:
totVarianceExplained[-1]

139 components explain 78% of the variance. That's a lot of components but still we managed to reduce our dataframe by half the features (278 initially), not bad right?!

## **K-means clustering**

K-means clustering allows us to divide our dataset into a defined number of clusters (k) by assigning each observation to the cluster with the nearest mean.

We combine k-means clustering with PCA to increase the data segmentation of our results. PCA reduced the number of features to fewer uncorrelated ones, reducing the noise in the data which can now be grouped more easily.

To simplify the computation, we reduce our dataframe to two components in our application.

Similarly to PCA, perform k-means clustering in `sklearn` we use a combination of the functions `KMeans()` and `fit()`.

In [ ]:
inertias = []
Nclusters = range(1,10)
for i in Nclusters:
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(prComp)
    inertias.append(kmeans.inertia_)

Similarly to PCA we can use *elbow method* to determine how many clusters we should keep. That is, we look for the *elbow* or the place where the inertia begins to slow.

In [ ]:
plt.figure(dpi=200)
plt.plot(Nclusters, inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.xticks(Nclusters[1::4])
plt.ylabel('Inertia')
plt.show()

In this case, 3 seems the right number of clusters to use.

Let's now combine the two methods and take a look at our clusters.

In [ ]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(prComp)

We define the clusters centroids representing the average of each cluster.

In [ ]:
centroids = kmeans.cluster_centers_
print("Cluster Centroids:\n", centroids)

In [ ]:
plt.figure(dpi=200)
plt.scatter(prComp[:,0], prComp[:,1], c= kmeans.labels_)
plt.scatter(centroids[:, 0], centroids[:, 1], marker="X", s=200, color="red", label="Centroids")
plt.xlabel('PC1 [-]')
plt.ylabel('PC2 [-]')
plt.show()

Our clusters are somehow separated (i.e., minimal overal is present) and centroids are apart from each other and located at the center of each cluster.

Another way to check if clustering is successful is to compute its Silhouette score, measuring how well a point fits within its assigned cluster.

* Silhouette Score = 1: Well-clustered (tight clusters, good separation)
* Silhouette Score = 0: Overlapping clusters (unclear boundary)
* Silhouette Score = -1: Wrong clustering (point assigned to the wrong group)

In [ ]:
sil_score = silhouette_score(prComp, kmeans.labels_)
sil_score

As expected, our score shows some overlap as the value is closer to 0 than to 1.